# Wetlands detection

Author: Morgane Magnier (morgane.magnier@vattenfall.com)

Copyright © 2024 Magnier Morgane 

This notebook is part of a thesis project. The copyright of the thesis itself belongs to the student Morgane Magnier.  

**Rights and Intellectual Property**:  
- Vattenfall has the right to use the findings, methods, and conclusions of this thesis in its operations.  
- Any material generated within the framework of this thesis that is subject to intellectual property protection (e.g., source code, computer program, design, or invention) belongs to Vattenfall, unless otherwise agreed in writing.  

Permission is granted to view, copy, and share this notebook for **educational or personal purposes only**, provided that this notice is included in all copies.  

---

In [7]:
import ee, eemont
import geemap
import math
import pandas as pd

import sys
sys.path.append('../../preprocessing/clouds')
import s2_preprocessing

import matplotlib.pyplot as plt
ee.Authenticate()
ee.Initialize()

# Sentinel 1 collection

In [91]:
roi =  ee.Geometry.Polygon([[[17.204933,60.402663],[17.204933,60.455525],[17.2645,60.455525],[17.2645,60.402663],[17.204933,60.402663]]])

start_date = ee.Date('2020-01-01')
end_date = ee.Date('2020-07-01')

s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(start_date, end_date).filterBounds(roi)

# Filter the Sentinel-1 collection by metadata properties.
vv_vh_iw = (
    s1
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
)

# Separate ascending and descending orbit images into distinct collections.
vv_vh_iw_asc = vv_vh_iw.filter(
    ee.Filter.eq('orbitProperties_pass', 'ASCENDING')
)
vv_vh_iw_desc = vv_vh_iw.filter(
    ee.Filter.eq('orbitProperties_pass', 'DESCENDING')
)

display(vv_vh_iw_desc)
display(vv_vh_iw_asc)

# Sentinel 2 collection

In [68]:
s2 = s2_preprocessing.get_s2_cloud_free_col_dates(roi, 0.95, '2018-03-01','2018-03-15').first() 
display(s2)

# GD collection

In [69]:
def filter_col(col, roi, band, thresh):
    
    col = col.map(lambda image : image.clip(roi))

    def count_pixels(image,roi): 
        pixel_count = image.select(band).reduceRegion(
            reducer= ee.Reducer.count(),
            geometry=roi,
            scale=10,
            maxPixels=1e9
        ).get(band)
        return image.set('pixel_count', pixel_count)

    nb_pixels_ts = col.map(lambda image: count_pixels(image, roi))

    # Get the image with the maximum pixel count
    max_pixel_count_image = nb_pixels_ts.sort('pixel_count', False).first()
    ref_img_pixel_count = max_pixel_count_image.get('pixel_count').getInfo()
    pixel_count_threshold = ref_img_pixel_count * thresh

    # Filter the collection based on the pixel count threshold
    filtered_col = nb_pixels_ts.filter(ee.Filter.gte('pixel_count', pixel_count_threshold))

    return filtered_col

def get_gd_cloud_free_col(roi, thresh):
    
    gd = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterBounds(roi)
    gd = filter_col(gd,roi,'label',thresh)
    
    return gd

def reclassify_image(image):
    label = image.select('label')
    
    # Créer des masques pour les classes "water" (0) et "snow_and_ice" (8)
    water_class = label.eq(0)  # Classe 0 : water
    snow_and_ice_class = label.eq(8)  # Classe 8 : snow_and_ice
    
    # Créer une classe "other" pour toutes les autres valeurs
    other_class = label.neq(0).And(label.neq(8))  # Toutes les autres classes

    # Assigner des valeurs pour chaque classe
    classified = water_class.multiply(1).add(snow_and_ice_class.multiply(2)).add(other_class.multiply(3))

    return classified.rename('label').copyProperties(image,['system:time_start'])

In [106]:
class_colors = {
    'water': '#419bdf',
    'snow_and_ice': '#b39fe1',
    'other': '#a59b8f'
}

# Définir les valeurs des classes et les couleurs correspondantes
class_values = list(range(3))
class_palette = list(class_colors.values())

# Définir les paramètres de visualisation
vis_params = {
    'min': 1,
    'max': 3,
    'palette': class_palette
}

s2 = s2_preprocessing.get_s2_cloud_free_col_dates(roi, 0.95, '2020-01-01','2020-02-01').median() 
display(s2)

gd = get_gd_cloud_free_col(roi, 0.5).filterDate('2020-01-01','2020-02-01').first()
gd = ee.Image(reclassify_image(gd))

display(gd)

result = ee.Image(vv_vh_iw_asc.filterDate('2020-01-01','2020-02-01').toList(vv_vh_iw_asc.size()).get(1)).clip(roi)
display(result)

s2_rgbVis = {'bands': ['B4', 'B3', 'B2'],'min': 0.0,'max': 1}
rgb = ee.Image.rgb(result.select('VV'),result.select('VH'),result.select('VV').divide(result.select('VH')))
map = geemap.Map()
map.centerObject(result, zoom=13)
map.addLayer(gd.select('label').clip(roi), vis_params, 'label')
map.addLayer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')
map.addLayer(result.select('VV'),  {'min': -25,'max': -5}, 'VV')
map.addLayer(result.select('VH'),  {'min': -35,'max': -10}, 'VH')
map.addLayer(s2.divide(10000), s2_rgbVis, 's2')
map 

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [107]:
import geemap.chart as chart

s1_img = result

samples = s1_img.sample(
        numPixels=10000,
        #classBand='label', 
        scale=10, 
        region = roi
    )

property = 'VV'

options = {
    "title": "Histogramm of VV band for S1 image,january 2020",
    "xlabel": "VV band value (dB)",
    "ylabel": "Pixel count",
    "colors": ["#1d6b99"],
}

chart.feature_histogram(samples, property, **options)


In [108]:
def add_dates(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    return image.set({'date': date})

def find_matching_images(collection, gd_collection):
    
    def find_matching_image(image):
        date = ee.Date(image.get('system:time_start'))
        matched_gd_image = gd_collection.filterDate(date, date.advance(1, 'day')).first()
        return ee.Algorithms.If(
            matched_gd_image,
            image.addBands(matched_gd_image.select('label')),
            None
        )

    matched_images = collection.map(find_matching_image, dropNulls=True)
    return ee.ImageCollection(matched_images)

In [117]:
def radarClassifier (input, region = roi, scale = 30 , clusterNumber = 2):
    training = input.sample(region = region,scale = scale,numPixels = 10000) #geometry ??
    clusterer = ee.Clusterer.wekaKMeans(clusterNumber, distanceFunction="Euclidean", maxIterations=75).train(training)
    return input.addBands(input.cluster(clusterer))

def identify_glace_or_other(image):
    # Obtenir les fréquences des clusters
    cluster_histogram = image.select('cluster').reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=roi,
        scale=30,
        maxPixels=1e10
    )

    # Extraire les identifiants des clusters (les clés de l'histogramme)
    cluster_ids = ee.Dictionary(cluster_histogram.get('cluster')).keys()

    # Fonction pour calculer les moyennes VV et VH pour un cluster donné
    def compute_cluster_means(cluster):
        cluster = ee.Number.parse(cluster)  # Convertir l'identifiant du cluster en nombre
        cluster_mask = image.select('cluster').eq(cluster)
        cluster_image = image.updateMask(cluster_mask)
        mean_dict = cluster_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi,
            scale=30,
            maxPixels=1e10
        )
        return ee.Dictionary({
            'cluster': cluster,
            'mean_VV': mean_dict.get('VV'),
            'mean_VH': mean_dict.get('VH')
        })

    # Appliquer la fonction pour chaque cluster
    clusters_means = cluster_ids.map(compute_cluster_means)

    # Convertir en listes pour faciliter la comparaison
    cluster1 = ee.Dictionary(clusters_means.get(0))
    cluster2 = ee.Dictionary(clusters_means.get(1))

    mean_vv_1 = ee.Number(cluster1.get('mean_VV'))
    mean_vv_2 = ee.Number(cluster2.get('mean_VV'))

    # Identifier quel cluster est glace
    ice_cluster = ee.Number(ee.Algorithms.If(mean_vv_1.gt(mean_vv_2), cluster1.get('cluster'), cluster2.get('cluster')))
    other_cluster = ee.Number(ee.Algorithms.If(mean_vv_1.lt(mean_vv_2), cluster1.get('cluster'), cluster2.get('cluster')))

    # Convertir les clusters en images pour le masquage
    ice_cluster_image = ee.Image.constant(ice_cluster)
    other_cluster_image = ee.Image.constant(other_cluster)

    # Créer les images séparées
    ice_image = image.select('cluster').eq(ice_cluster_image).rename('ice').updateMask(image.select('cluster').eq(ice_cluster_image))
    other_image = image.select('cluster').eq(other_cluster_image).rename('else').updateMask(image.select('cluster').eq(other_cluster_image))

    return ice_image, other_image

def ice_water_discrimination_1(image):
    ice_snow_else = image.select('VV').gt(-13.7).And(image.select('VH').gt(-21.2))
    image_to_classify = image.updateMask(ice_snow_else)
    image_classified = radarClassifier (image_to_classify)
    ice, other = identify_glace_or_other(image_classified)
    ice_water = (image.select('VV').gt(-13.7).And(image.select('VH').lt(-21.2))).Or(image.select('VV').lt(-13.7).And(image.select('VH').gt(-21.2)))
    water = image.select('VV').lt(-13.7).And(image.select('VH').lt(-21.2))

    # Étape 4: Créer une image de classification combinée
    classified_image = ee.Image(0)  # Initialiser avec des pixels à 0 (classe par défaut 'water')

    # Ajouter les classes avec les valeurs respectives
    classified_image = classified_image.where(water, 0)
    classified_image = classified_image.where(ice, 1)
    classified_image = classified_image.where(ice_water, 2)
    classified_image = classified_image.where(other, 3)
    
    # Étape 5: Retourner l'image classifiée finale
    return classified_image.rename('classification')

def ice_water_discrimination_2(image):
    ice_snow_else = (image.select('VH').gt(-20.2))#image.select('VV').gt(-13.7)
    image_to_classify = image.updateMask(ice_snow_else)
    image_classified = radarClassifier (image_to_classify)
    ice, other = identify_glace_or_other(image_classified)
    ice_water = (image.select('VV').gt(-13.7).And(image.select('VH').lt(-21.2))).Or(image.select('VV').lt(-13.7).And(image.select('VH').gt(-21.2)))
    water = image.select('VH').lt(-20.2)#image.select('VV').lt(-13.7).And(

    # Étape 4: Créer une image de classification combinée
    classified_image = ee.Image(0)  # Initialiser avec des pixels à 0 (classe par défaut 'water')

    # Ajouter les classes avec les valeurs respectives
    classified_image = classified_image.where(water, 0)
    classified_image = classified_image.where(ice, 1)
    classified_image = classified_image.where(ice_water, 2)
    classified_image = classified_image.where(other, 3)
    
    # Étape 5: Retourner l'image classifiée finale
    return classified_image.rename('classification')

def ice_water_discrimination_3(image):
    other = (image.select('VV').gt(-18.25)).selfMask()#image.select('VV').gt(-13.7)
    ice = (image.select('VV').gt(-19.5).And(image.select('VV').lt(-18.25))).selfMask()
    water = image.select('VV').lt(-19.5).selfMask()#image.select('VV').lt(-13.7).And(

    # Étape 4: Créer une image de classification combinée
    # # Initialiser avec des pixels à 0 (classe par défaut 'water')

    # Ajouter les classes avec les valeurs respectives
    classified_image = classified_image.where(water, 0)
    classified_image = classified_image.where(ice, 1)
    classified_image = classified_image.where(other, 2)
    
    # Étape 5: Retourner l'image classifiée finale
    return classified_image.rename('classification')

In [118]:
#result = ee.Image(vv_vh_iw_asc.filterDate('2018-02-01','2018-03-01').toList(vv_vh_iw_asc.size()).get(0)).clip(roi)

class_colors = {
    'water': '#419bdf',      # bleu pour l'eau
    'snow_and_ice': '#b39fe1', # violet pour la glace
    #'ice_water': '#e1b39f',  # orange clair pour glace/eau mixte
    'other': '#a59b8f'       # gris pour autres
}

# Créer la palette de couleurs dans l'ordre des classes
class_palette = [class_colors['water'], 
                 class_colors['snow_and_ice'], 
                 #class_colors['ice_water'], 
                 class_colors['other']]

# Définir les paramètres de visualisation
vis_params = {
    'min': 0,
    'max': 2,
    'palette': class_palette
}

# Supposons que 'classified_image' est l'image classifiée que vous avez créée
classified_image = ice_water_discrimination_3(result.clip(roi))

# Créer une carte et afficher l'image classifiée avec les couleurs spécifiées
Map = geemap.Map()
Map.centerObject(roi, 10)  # Centrer la carte sur la région d'intérêt (roi)
Map.addLayer(classified_image.clip(roi), vis_params, 'Classified Image')

# Ajouter des contrôles à la carte
Map.add_colorbar(vis_params)

# Afficher la carte
Map

UnboundLocalError: cannot access local variable 'classified_image' where it is not associated with a value